In [1]:
from collections import Counter
import pickle
import numpy as np
import subprocess

In [2]:
with open('intersection.pickle','rb') as f1:
    intersection=pickle.load(f1)
with open('union.pickle','rb') as f2:
    union=pickle.load(f2)
with open('difference.pickle','rb') as f3:
    difference=pickle.load(f3)

In [3]:
en_corpus=[]
es_corpus=[]
with open('dev.en') as f1:
    while True:
        en = f1.readline()
        if en =='' or en is None:
            break
        en_corpus.append(len(['NULL']+[w for w in en.strip().split()]))
        
with open('dev.es') as f2:
    while True:
        es = f2.readline()
        if es=='' or es is None:
            break
        es_corpus.append(len([w for w in es.strip().split()]))

In [4]:
def alignment_to_file(alignment,fname):
    
    output = [list(i) for i in sorted(list(alignment))]
    with open(fname,'w') as f1:
        for a in output:
            f1.write(' '.join([str(j) for j in a])+'\n')

In [17]:
alignment_to_file(union,'dev.keys.union')
alignment_to_file(intersection,'dev.keys.intersection')
alignment_to_file(difference,'dev.keys.difference')

In [20]:
!python3 eval_alignment.py dev.key dev.keys.union

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.363        0.565        0.442


In [22]:
!python3 eval_alignment.py dev.key dev.keys.intersection

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.775        0.352        0.485


In [23]:
!python3 eval_alignment.py dev.key dev.keys.difference

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.193        0.213        0.203


In [5]:
with open('intersection.pickle','rb') as f1:
    intersection=pickle.load(f1)
with open('union.pickle','rb') as f2:
    union=pickle.load(f2)
with open('difference.pickle','rb') as f3:
    difference=pickle.load(f3)
new_alignment=intersection.copy()
c=0
for iterations in range(10):
    counter=0
    for k in range(1,len(en_corpus)+1):
        m = en_corpus[k-1]-1
        l = es_corpus[k-1]
        mat = np.zeros( (m+2,l+2) )
        new_set=set()
        new_set2=set()
        for j in range(1,m+1):
            for i in range(1,l+1):
                if (k,j,i) in new_alignment:
                    new_set.add((k,j,i))
                    mat[j,i] = 1
                elif (k,j,i) in difference:
                    new_set2.add((k,j,i))
                    mat[j,i] = -1
#         print("Elements added as 1: ",new_set)
#         print("Elements added as -1: ",new_set2)
#         if k>10:
#             break
        for j in range(1,m+1):
            for i in range(1,l+1):
                if mat[j,i] == -1:
                    if np.any(mat[j-1:j+2,i-1:i+2]==1):
                        mat[j,i]=1
                        counter+=1
                        new_alignment.add((k,j,i))
                        difference.remove((k,j,i))
    print("Iteration {}\n".format(iterations+1))
    if counter !=0:
        print("Elements added: {}\n".format(counter))
        fname = 'dev.keys.iteration_'+str(iterations)
        alignment_to_file(new_alignment,fname)
        returned_output = subprocess.check_output("python3 eval_alignment.py dev.key " + fname,shell=True)
        print(returned_output.decode('utf-8'))

        with open('./output_pickle_files/new_alignment_step_'+str(iterations)+'.pickle','wb') as f1:
            pickle.dump(new_alignment,f1)
    else:
        print("No new alignments found. Exiting")
        break

Iteration 1

Elements added: 1289

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.673        0.453        0.541

Iteration 2

Elements added: 168

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.662        0.464        0.545

Iteration 3

Elements added: 44

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.659        0.466        0.546

Iteration 4

Elements added: 10

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.658        0.467        0.547

Iteration 5

Elements added: 4

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.658        0.468        0.547

Iteration 6

Elements added: 2

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.658        0.468        0.547

Iteration 7

Elements added: 1

      Type       Total   Precision      Recall   

In [21]:
with open('intersection.pickle','rb') as f1:
    intersection=pickle.load(f1)
with open('union.pickle','rb') as f2:
    union=pickle.load(f2)
with open('difference.pickle','rb') as f3:
    difference=pickle.load(f3)
new_alignment=intersection.copy()
c=0
for iterations in range(10):
    counter=0
    for k in range(1,len(en_corpus)+1):
        m = en_corpus[k-1]-1
        l = es_corpus[k-1]
        mat = np.zeros( (m+2,l+2) )
        new_set=set()
        new_set2=set()
        for j in range(1,m+1):
            for i in range(1,l+1):
                if (k,j,i) in new_alignment:
                    new_set.add((k,j,i))
                    mat[j,i] = 1
                elif (k,j,i) in difference:
                    new_set2.add((k,j,i))
                    mat[j,i] = -1
#         print("Elements added as 1: ",new_set)
#         print("Elements added as -1: ",new_set2)
#         if k>10:
#             break
        for j in range(1,m+1):
            for i in range(1,l+1):
                if mat[j,i] == -1:
                    if np.any(mat[j-1:j+2,i-1:i+2]==1):
                        mat[j,i]=1
                        counter+=1
                        new_alignment.add((k,j,i))
                        difference.remove((k,j,i))
    print("Iteration {}\n".format(iterations+1))
    if counter !=0:
        print("Elements added: {}\n".format(counter))
        fname = 'dev.keys.iteration_'+str(iterations)
        alignment_to_file(new_alignment,fname)
        returned_output = subprocess.check_output("python3 eval_alignment.py dev.key " + fname,shell=True)
        print(returned_output.decode('utf-8'))

        with open('./output_pickle_files/new_alignment_step_'+str(iterations)+'.pickle','wb') as f1:
            pickle.dump(new_alignment,f1)
    else:
        print("No new alignments found. Exiting")
        break

Iteration 1

Elements added: 821

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.700        0.415        0.521

Iteration 2

Elements added: 115

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.690        0.423        0.524

Iteration 3

Elements added: 33

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.687        0.425        0.525

Iteration 4

Elements added: 7

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.687        0.425        0.525

Iteration 5

Elements added: 3

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.687        0.426        0.526

Iteration 6

Elements added: 1

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.687        0.426        0.526

Iteration 7

Elements added: 1

      Type       Total   Precision      Recall     

In [102]:
# counter=0
# for iterations in range(10):
#     for k in range(len(en_corpus)):
#         for j in range(1,en_corpus[k]):
#             for i in range(1,es_corpus[k]+1) :
#                 if (k,j,i) not in new_alignment and ((k,j,i-1) in union or (k,j,i+1) in union) and (k,j,i) in union:
#                     new_alignment.add((k,j,i))
#                     if counter % steps == 0:
#                         with open('./output_pickle_files/new_alignment2_step_'+str(counter/steps)+'.pickle','wb') as f1:
#                             pickle.dump(new_alignment,f1)
#                     counter+=len(new_set)

In [108]:
# counter=0
# steps=10
# for iterations in range(10):
#     for k in range(len(en_corpus)):
#         for j in range(1,en_corpus[k]):
#             for i in range(1,es_corpus[k]+1) :
#                 if (k,j,i) not in new_alignment and (k,j,i) in union:
#                     new_alignment.add((k,j,i))
#                     if counter % steps == 0:
#                         with open('./output_pickle_files/new_alignment3_step_'+str(counter/steps)+'.pickle','wb') as f1:
#                             pickle.dump(new_alignment,f1)
#                     counter+=len(new_set)

In [62]:
# counter=0
# for k in range(len(en_corpus)):
#     for i in range(1,es_corpus[k]+1):
#         for j in range(1,en_corpus[k]):
#             if (k,j,i) not in new_alignment and (k,j,i) in union:
#                     new_alignment.add((k,j,i))
#                     if counter % steps == 0:
#                         with open('./output_pickle_files/new_alignment_step_'+str(counter/steps)+'.pickle','wb') as f1:
#                             pickle.dump(new_alignment,f1)
#                     counter+=1

In [32]:
# print(len(new_alignment),len(union),len(difference),len(intersection))

In [33]:
# print(counter)
# print(steps)

In [98]:
# def pickle_to_key_file(pickle_file):
#     with open('./output_pickle_files/'+pickle_file+'.pickle','rb') as f1:
#         keys_set=pickle.load(f1)
#         outputs=sorted(list(keys_set))
#         with open('./output_key_files/'+pickle_file+'.key','w') as f2:
#             for k in outputs:
#                 s=str(k[0])+' '+str(k[1])+ ' ' + str(k[2])+'\n'
#                 f2.write(s)

In [111]:
# for i in range(1):
#     pickle_to_key_file('new_alignment3_step_'+str(float(i)))

In [22]:

# for i in range(1):
#     print(i)
#     returned_output = subprocess.check_output("python3 eval_alignment.py dev.key ./output_key_files/new_alignment3_step_"+str(i)+".0.key",shell=True)
#     print(returned_output.decode('utf-8'))

In [23]:
# import subprocess
# for i in range(1):
#     print(i)
#     returned_output = subprocess.check_output("python3 eval_alignment.py dev.key ./output_key_files/new_alignment2_step_"+str(i)+".0.key",shell=True)
#     print(returned_output.decode('utf-8'))

In [24]:
# import subprocess
# for i in range(3):
#     print(i)
#     returned_output = subprocess.check_output("python3 eval_alignment.py dev.key ./output_key_files/new_alignment1_step_"+str(i)+".0.key",shell=True)
#     print(returned_output.decode('utf-8'))

In [25]:
# !python3 eval_alignment.py dev.key new_alignment_step_0.0.key

In [26]:
# !python3 eval_alignment.py dev.key new_alignment_step_1.0.key

In [27]:
# !python3 eval_alignment.py dev.key new_alignment_step_2.0.key

In [28]:
# !python3 eval_alignment.py dev.key new_alignment_step_3.0.key

In [29]:
# !python3 eval_alignment.py dev.key new_alignment_step_4.0.key

In [30]:
# !python3 eval_alignment.py dev.key new_alignment_step_5.0.key